## Training YOLOv9 on the BRIDGE dataset

Ensure access to NVIDIA gpu withing colab environment

In [1]:
!nvidia-smi

Sat Jun 29 07:41:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**NOTE:** Creating a `HOME` constant to helpmanada dataset, images and model paths

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


# Cloning YOLOv9 github repo and installations

Since YOLOv9 is new, use a fork of the github repo. There is a bug in the `detect.py` file which has been fixed in the fork.

In [3]:
!git clone https://github.com/Krisha-11/yolov9-fork.git
%cd yolov9-fork
!pip install -r requirements.txt -q

Cloning into 'yolov9-fork'...
remote: Enumerating objects: 502, done.
remote: Counting objects: 100% (359/359), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 502 (delta 269), reused 244 (delta 226), pack-reused 143
Receiving objects: 100% (502/502), 2.61 MiB | 24.71 MiB/s, done.
Resolving deltas: 100% (275/275), done.
/content/yolov9-fork
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 63.0 MB/s eta 0:00:00


**NOTE:** Install the roboflo package to directly download the dataset.

In [4]:
!pip install -q roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.2 MB/s eta 0:00:00


# Dowloading model's weights from official github repo

In [5]:
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [6]:
!ls -la {HOME}/weights

total 402440
drwxr-xr-x 2 root root      4096 Jun 29 07:43 .
drwxr-xr-x 1 root root      4096 Jun 29 07:43 ..
-rw-r--r-- 1 root root  51508261 Feb 18 12:36 gelan-c.pt
-rw-r--r-- 1 root root 117203713 Feb 18 12:36 gelan-e.pt
-rw-r--r-- 1 root root 103153312 Feb 18 12:36 yolov9-c.pt
-rw-r--r-- 1 root root 140217688 Feb 18 12:36 yolov9-e.pt


# Downloading dataset from Roboflow

Ensure dataset is saved in the `{HOME}/yolov9-fork` directory to successfully train model with it

In [7]:
%cd {HOME}/yolov9-fork

/content/yolov9-fork


In [8]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="w0cyD276pmUe8RU1k2RV")
project = rf.workspace("ddp-6p5k9").project("bridge-lclml")
version = project.version(7)
dataset = version.download("yolov9")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to BRIDGE-7 in yolov9:: 100%|██████████| 6722/6722 [00:01<00:00, 5056.03it/s]


# Train Custom Model

In [9]:
%cd {HOME}/yolov9-fork

/content/yolov9-fork


In [10]:
!python train.py \
--batch 16 --epochs 25 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data {dataset.location}/data.yaml \
--weights {HOME}/weights/gelan-c.pt \
--cfg models/detect/gelan-c.yaml \
--hyp hyp.scratch-high.yaml

2024-06-29 07:43:43.223166: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 07:43:43.223218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 07:43:43.329582: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-29 07:43:43.397435: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-29 07:43:45.050408: W tensorflow/comp

In [11]:
!ls # ! creates a temporary terminal and executes the bash commands there, thus once the command is done, it resets to the original, so doing cd .. doesnt move the directory backward, thus use % that connects with main terminal

benchmarks.py	detect.py   models	      runs	     train.py	      val_triple.py
BRIDGE-7	export.py   panoptic	      scripts	     train_triple.py
classify	figure	    __pycache__       segment	     utils
data		hubconf.py  README.md	      tools	     val_dual.py
detect_dual.py	LICENSE.md  requirements.txt  train_dual.py  val.py


In [14]:
%cd ..

/content


In [15]:
%ls

sample_data/  weights/  yolov9-fork/


In [16]:
!zip -r weights.zip weights/
!zip -r yolov9-fork.zip yolov9-fork/

Streaming output truncated to the last 5000 lines.
  adding: yolov9-fork/BRIDGE-7/train/images/000198_jpg.rf.46657b318ac6f463779b65fc6efb66c2.jpg (deflated 35%)
  adding: yolov9-fork/BRIDGE-7/train/images/000090_jpg.rf.3ead537cb406727931fd9f3aaa14a6f0.jpg (deflated 22%)
  adding: yolov9-fork/BRIDGE-7/train/images/221061_jpg.rf.73361ba972b8a7e2a3ab793b9cdd4360.jpg (deflated 37%)
  adding: yolov9-fork/BRIDGE-7/train/images/000403_jpg.rf.5e2be4bea901c4ff067640443e3c66f8.jpg (deflated 37%)
  adding: yolov9-fork/BRIDGE-7/train/images/Cat38_3_jpg.rf.69ef0b699bb8b2c133f2be0b94069087.jpg (deflated 44%)
  adding: yolov9-fork/BRIDGE-7/train/images/116152_jpg.rf.b6a188f42351b8f10986a2d5816af575.jpg (deflated 7%)
  adding: yolov9-fork/BRIDGE-7/train/images/215481_jpg.rf.e8ca6b35a610e236a5620038e9c8c0ba.jpg (deflated 5%)
  adding: yolov9-fork/BRIDGE-7/train/images/000308_jpg.rf.33609c5f9afd869dfec9ed6230226586.jpg (deflated 28%)
  adding: yolov9-fork/BRIDGE-7/train/images/000453_jpg.rf.ae8ae304ed1a

In [17]:
from google.colab import files
files.download('weights.zip')
files.download('yolov9-fork.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>